In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [26]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.381 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [27]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [36]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('json', data_files='dataset/dataset_formatted.json', split='train')

In [37]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/1003 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

In [38]:
dataset[5]["conversations"]

[{'content': 'Sen doğal bir dil ile girilmiş ürün başlıklarını profesyonel, dikkat çekici ve satışa hazır hale getirmek için özelleştirilmiş bir pazarlama botusun. Kullanıcının girdiği satış açıklamalarını profesyonel hale getir. Ayrıca eğer kullanıcı girdisi yöresel kelime içeriyor ise bunun anlamını da aşağıdaki metinde bulabilirsin. Açıklamalarını global ve profesyonel bir hale çevirirken bu kelimeleri ve açıklamalarını da dikkate al.',
  'role': 'system'},
 {'content': 'Kullanıcı inputu:\n"Hatay\'dan 5 kg organik kırma zeytin"\n\nYöresel kelimeler:\n{}',
  'role': 'user'},
 {'content': '"Profesyonel Başlık": "Hatay\'dan Organik 5KG Kırma Zeytin - Yeni Sezon", \n"Profesyonel Açıklama": "Yeni sezon Hatay kırma zeytinleri, doğal yöntemlerle işlenmiş ve %100 organik olarak üretilmiştir. Salatalarınızda ve kahvaltılarınızda sağlıklı ve lezzetli bir alternatif sunar."',
  'role': 'assistant'}]

In [39]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nSen doğal bir dil ile girilmiş ürün başlıklarını profesyonel, dikkat çekici ve satışa hazır hale getirmek için özelleştirilmiş bir pazarlama botusun. Kullanıcının girdiği satış açıklamalarını profesyonel hale getir. Ayrıca eğer kullanıcı girdisi yöresel kelime içeriyor ise bunun anlamını da aşağıdaki metinde bulabilirsin. Açıklamalarını global ve profesyonel bir hale çevirirken bu kelimeleri ve açıklamalarını da dikkate al.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKullanıcı inputu:\n"Hatay\'dan 5 kg organik kırma zeytin"\n\nYöresel kelimeler:\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"Profesyonel Başlık": "Hatay\'dan Organik 5KG Kırma Zeytin - Yeni Sezon", \n"Profesyonel Açıklama": "Yeni sezon Hatay kırma zeytinleri, doğal yöntemlerle işlenmiş ve %100 organik olarak üretilmiştir. Salatalarınızda ve kahvaltılarınızda sağlıklı 

In [41]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [42]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

In [43]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nSen doğal bir dil ile girilmiş ürün başlıklarını profesyonel, dikkat çekici ve satışa hazır hale getirmek için özelleştirilmiş bir pazarlama botusun. Kullanıcının girdiği satış açıklamalarını profesyonel hale getir. Ayrıca eğer kullanıcı girdisi yöresel kelime içeriyor ise bunun anlamını da aşağıdaki metinde bulabilirsin. Açıklamalarını global ve profesyonel bir hale çevirirken bu kelimeleri ve açıklamalarını da dikkate al.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKullanıcı inputu:\n"Hatay\'dan 5 kg organik kırma zeytin"\n\nYöresel kelimeler:\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"Profesyonel Başlık": "Hatay\'dan Organik 5KG Kırma Zeytin - Yeni Sezon", \n"Profesyonel Açıklama": "Yeni sezon Hatay kırma zeytinleri, doğal yöntemlerle işlenmiş ve %100 organik olarak üretilmiştir. Salatalarınızda ve kahvaltılarınızda sağlıklı 

In [44]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                       \n\n"Profesyonel Başlık": "Hatay\'dan Organik 5KG Kırma Zeytin - Yeni Sezon", \n"Profesyonel Açıklama": "Yeni sezon Hatay kırma zeytinleri, doğal yöntemlerle işlenmiş ve %100 organik olarak üretilmiştir. Salatalarınızda ve kahvaltılarınızda sağlıklı ve lezzetli bir alternatif sunar."<|eot_id|>'

In [45]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,003 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 310
 "-____-"     Number of trainable parameters = 97,255,424


Step,Training Loss
1,1.839700
2,1.793800
3,1.863500
4,1.808600
5,1.884000
6,1.908400
7,1.859800
8,1.833500
9,1.842100
10,1.861600


In [54]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# {"role": "system", "content": "Sen doğal bir dil ile girilmiş ürün başlıklarını profesyonel, dikkat çekici ve satışa hazır hale getirmek için özelleştirilmiş bir pazarlama botusun. Kullanıcının girdiği satış açıklamalarını profesyonel hale getir. Ayrıca eğer kullanıcı girdisi yöresel kelime içeriyor ise bunun anlamını da aşağıdaki metinde bulabilirsin. Açıklamalarını global ve profesyonel bir hale çevirirken bu kelimeleri ve açıklamalarını da dikkate al. Cevap olarak: \"\"\"\n\"Profesyonel Başlık\": ...\n\"Profesyonel Açıklama\": ...\n\"\"\" şablonunu kullan. Eğer kullanıcı uzun ve profesyonel olmayan ek açıklamalar girdiyse bunları \"Profesyonel Başlık\" kısmında gösterme ancak \"Profesyonel Açıklama\" kısmına ekle. "},
messages = [
    {"role": "system", "content": "You are a marketing bot that is specialized to make natural language product titles professional, eye-catching and sales-ready. Make user-entered sales descriptions professional. Also, if the user input contains regional words, you can find the meaning of this in the text below. When translating your descriptions into a global and professional form, take these words and their descriptions into account. In response: Use the template \"\"\"\n\"Profesyonel Başlık\": ...\n\"Profesyonel Açıklama\": ...\n\"\"\". If the user enters long and unprofessional additional descriptions, do not display them in the \"Profesyonel Başlık\" section, but add them to the \"Profesyonel Açıklama\" section."},
    {"role": "user", "content": "Kullanıcı inputu: \"700
     gram kuşburnu marmelat yüzde yüz dogal açtıktan sonra dolaba koyun\" Yöresel kelimeler: {}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 0.1, min_p = 0.5)

"Profesyonel Başlık": "700g Doğal Kuşburnu Marmelat - %100 Doğal", 
"Profesyonel Açıklama": "700 gram kuşburnu marmelat, %100 doğal yöntemlerle hazırlanmıştır. Doğal olarak hazırlanan bu marmelat, doğal tatları ile lezzetli bir alternatif sunar. Dolabınıza koyun ve sağlıklı bir kahvaltı sunun."<|eot_id|>


In [55]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [57]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a marketing bot that is specialized to make natural language product titles professional, eye-catching and sales-ready. Make user-entered sales descriptions professional. Also, if the user input contains regional words, you can find the meaning of this in the text below. When translating your descriptions into a global and professional form, take these words and their descriptions into account. In response: Use the template \"\"\"\n\"Profesyonel Başlık\": ...\n\"Profesyonel Açıklama\": ...\n\"\"\". If the user enters long and unprofessional additional descriptions, do not display them in the \"Profesyonel Başlık\" section, but add them to the \"Profesyonel Açıklama\" section."},
    {"role": "user", "content": "Kullanıcı inputu: \"700 gram kuşburnu marmelat yüzde yüz dogal açtıktan sonra dolaba koyun\" Yöresel kelimeler: {}"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 0.1, min_p = 0.5)

"Profesyonel Başlık": "700g Doğal Kuşburnu Marmelat - %100 Doğal", 
"Profesyonel Açıklama": "700 gram kuşburnu marmelat, %100 doğal yöntemlerle hazırlanmıştır. Doğal olarak hazırlanan bu marmelat, doğal tatları ile lezzetli bir alternatif sunar. Dolabınıza koyabilirsiniz."<|eot_id|>


In [58]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 53.53 out of 83.48 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 97.77it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
